In [18]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
a = (0.755+0.534+0.711) / 3
a

0.6666666666666666

In [4]:
# exp                       checkpoint
# 0001-res-dog6_CR          400
# 0002-res-cat2_CR          200
# 0003-res-duck_toy_CR      200
# 0004-res-vase_CR          800
# 0005-res-backpack_CR      200
# 0006-res-berry_bowl_CR    200
# 0007-res-backpack_dog_CR  600
# [a for a in os.listdir(base_path)] 

In [20]:
import regex 
import os
import random
import numpy as np
from matplotlib.ticker import FormatStrFormatter


import torch
import torch.backends.cuda

import matplotlib_inline
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')

# torch.backends.cuda.enable_math_sdp(True)
# torch.backends.cuda.enable_flash_sdp(False)
# torch.backends.cuda.enable_mem_efficient_sdp(False)

In [5]:
!pip install torch
!pip install scikit-learn
!pip install ipyfilechooser
!pip install natsort
!pip install seaborn

In [21]:
import sklearn

In [22]:
# {('0042-res-dog6_CR', ('500', '50', '7.0')): {'TS': 0.244346, 'BIS': 0.922852, 'IS': 0.893906}}


In [23]:
from nb_utils.clip_eval import aggregate_similarities

from nb_utils.cache import DistributedCache, Cache

from nb_utils.experiments_viewer import ExpsViewer
from nb_utils.clip_eval import ExpEvaluator
from nb_utils.images_viewer import MultifolderViewer
from nb_utils.configs import live_object_data

In [24]:
device = torch.device('cuda', 0) if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [25]:
torch.__version__

'1.7.1+cu101'

In [26]:
import sys
from IPython import get_ipython

ipython = get_ipython()

if ipython is not None:
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")
    ipython.magic("config Completer.use_jedi = True")
    ipython.magic("matplotlib inline")

import warnings
warnings.filterwarnings("ignore")

import regex
import os
import random
import numpy as np
from matplotlib.ticker import FormatStrFormatter


import torch
import torch.backends.cuda

import matplotlib_inline
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')

torch.backends.cuda.enable_math_sdp(True)
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_mem_efficient_sdp(False)

from nb_utils.clip_eval import aggregate_similarities

from nb_utils.cache import DistributedCache, Cache

from nb_utils.experiments_viewer import ExpsViewer
from nb_utils.clip_eval import ExpEvaluator
from nb_utils.images_viewer import MultifolderViewer
from nb_utils.configs import live_object_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'torch.backends.cuda' has no attribute 'enable_math_sdp'

In [8]:
evaluator = ExpEvaluator(device)

Downloading: "https://github.com/facebookresearch/dinov2/archive/master.zip" to /home/jovyan/.cache/torch/hub/master.zip


AttributeError: module 'torch' has no attribute 'inference_mode'

In [6]:
base_path = "/home/jovyan/shares/SR006.nfs2/chumachenko/diffusers/examples/core/res_CR/"
# exps = ["0-res-dog6_DB", "0-res-backpack_dog_DB", "0-res-berry_bowl_DB"]
exps = [a for a in os.listdir(base_path)]
exps_viewer = ExpsViewer(
    base_path=base_path,
    exp_filter_fn=lambda x: x in list(exps),
    ncolumns=4, lazy_load=True, evaluator=evaluator
)

stats = exps_viewer.evaluate(
    exps_names=exps, 
    checkpoint_idx='500', 
    inference_specs=('50', '7.0')
)
for key, value in stats.items():
    exp_cache = Cache(os.path.join(value['config']['output_dir'], 'eval.cache'))
    exp_cache.update({key: value})
    print('ready')

  0%|                                                                      | 0/3 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

 33%|████████████████████▋                                         | 1/3 [00:15<00:31, 15.83s/it]

  0%|          | 0/26 [00:00<?, ?it/s]

 67%|█████████████████████████████████████████▎                    | 2/3 [00:22<00:10, 10.71s/it]

  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████| 3/3 [00:30<00:00, 10.19s/it]

ready
ready
ready


In [7]:
ti_evaluate_cache = DistributedCache('/home/mdnikolaev/aschumachenko/diffusers/examples/dreambooth/res_DB/*/eval.cache')
# ti_evaluate_cache.keys()
# ti_evaluate_cache[key]
d = {k: {**v, **aggregate_similarities(v)} for k, v in ti_evaluate_cache.get().items()}
# d.keys()
# d[key]

In [8]:
def make_dict(path):
    ti_evaluate_cache = DistributedCache(path)
    d = {k: {**v, **aggregate_similarities(v)} for k, v in ti_evaluate_cache.get().items()}
    medium_base_res = {}
    for key in d.keys():
        if live_object_data[d[key]['config']['class_name']] == 'live':
            kt = 'live_text_similarity'
            ki = 'live_image_similarity'
        else:
            kt = 'object_text_similarity'
            ki = 'object_image_similarity'
            
        kb = 'base_image_similarity'
        if not d[key][kt] or not d[key][ki]:
            if live_object_data[d[key]['config']['class_name']] == 'live':
                kt = 'live_with_class_text_similarity'
                ki = 'live_with_class_image_similarity'
            else:
                kt = 'object_with_class_text_similarity'
                ki = 'object_with_class_image_similarity'
            kb = 'base_with_class_image_similarity'
        medium_base_res[key] = {
            'TS': np.round(d[key][kt], 6),
            'BIS': np.round(d[key][kb], 6),
            'IS': np.round(d[key][ki], 6),
        }
    return medium_base_res

In [10]:
res = make_dict('/home/mdnikolaev/aschumachenko/diffusers/examples/dreambooth/res_DB/*/eval.cache')

In [11]:
# запустив ячейку можно посмотреть сохраненные картинки, если они были правильно сохранены

ti_exps_viewer = ExpsViewer(
    # Base path to experiments
    '/home/mdnikolaev/aschumachenko/diffusers/examples/dreambooth/res_DB',
    # Filter experiments
    # exp_filter_fn=lambda x: x in [x for x in os.listdir('results') if "CD" not in x], 
    # Filter prompts
    # filter_fn='a photo of a sks',
    # Defines number of columns in experiment selector
    ncolumns=4, 
    # Whether to load all images only on demand
    lazy_load=True, 
    # ExpEvaluator instance to perform evaluation. See scripts/*/inference.sh
    evaluator=None,
)
ti_exps_viewer.view()

In [10]:
s1 = 'a'
s_list = ['a', 'b']
s_list = [(s1 + s) for s in s_list]
s_list
# for s in s_list:
#     s = s1 + s 
#     print(s)

['aa', 'ab']

In [8]:
s_list

['a', 'b']